In [6]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'..')

import numpy as np
import multiprocessing
from src.Sublists.overlap_worker import process_worker

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def add_subarray(A, r, sub=None):
    if sub is None:
        return np.random.choice(A, size=r, replace=False)
    else:
        return np.vstack((sub, np.random.choice(A, size=r, replace=False)))


def mp_intersections(sub, processes=4):
    num_rows = sub.shape[0]
    total_overlaps = 0
    result_queue = multiprocessing.Queue()

    chunk_size = num_rows // processes

    jobs = []
    for i in range(0, num_rows, chunk_size):
        j = min(i + chunk_size, num_rows)
        p = multiprocessing.Process(target=process_worker, args=(sub, i, j, result_queue))
        jobs.append(p)
        p.start()

    for p in jobs:
        p.join()

    while not result_queue.empty():
        total_overlaps += result_queue.get()

    return total_overlaps

In [11]:
n = 100
r = 20
STOP = 0.1
A = np.arange(1,n+1)
ipairs = []
inter = 0
H = 500
sub = None
processes = 8

while True:
    sub = add_subarray(A, r, sub)
    if sub.shape[0] % H == 0:
        inter = mp_intersections(sub, processes)
        print("Mean Intersection Rate: ", inter/sub.shape[0])
        if inter/sub.shape[0] > STOP:
            print(str(r)+"-sublist capacity of "+str(n)+": "+str(sub.shape[0]))
            break

Mean Intersection Rate:  0.022
Mean Intersection Rate:  0.043
Mean Intersection Rate:  0.059333333333333335
Mean Intersection Rate:  0.0845
Mean Intersection Rate:  0.1052
20-sublist capacity of 100: 2500
